In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from scipy.sparse import hstack, csr_matrix
import ast
import gc

In [7]:
# Chargement des données
df = pd.read_csv('C:/Users/pierr/Documents/Python/Projet_imdb/data/clean_final2.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29104 entries, 0 to 29103
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 29104 non-null  object 
 1   vote_average          29104 non-null  float64
 2   vote_count            29104 non-null  int64  
 3   release_date          29104 non-null  object 
 4   revenue               29104 non-null  int64  
 5   runtime               29104 non-null  int64  
 6   budget                29104 non-null  int64  
 7   imdb_id               29104 non-null  object 
 8   overview              29104 non-null  object 
 9   popularity            29104 non-null  float64
 10  poster_path           29104 non-null  object 
 11  genres                29104 non-null  object 
 12  production_companies  29104 non-null  object 
 13  keywords              29104 non-null  object 
 14  id_actor              29104 non-null  object 
 15  id_productor       

In [9]:
df["release_date"] = pd.to_datetime(df["release_date"])

In [10]:
# POIDS CONFIGURABLES
weights = {
    'text': 0.30,        # overview + keywords
    'genres': 0.25,     # genres
    'actors': 0.15,      # acteurs
    'directors': 0.20,  # producteurs
    'numeric': 0.1      # budget
}
    #'date" : 0.1       Possibilité (hésitation) de mettre les dates en petite pondération
print("Préparation des données...")

Préparation des données...


In [11]:
# Nettoyer les colonnes texte
df['overview'] = df['overview'].fillna('')
df['keywords'] = df['keywords'].fillna('')
df['actor_name'] = df['actor_name'].fillna('')
df['productor_name'] = df['productor_name'].fillna('')

print("Colonne nettoyé des NaN")    # Process à garder, aucun NaN n'était présent dans le Dataframe

Colonne nettoyé des NaN


In [12]:
# Préparer les genres (si c'est une string, la convertir en liste)
def parse_genres(x):
    if pd.isna(x) or x == '':
        return []
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x     # Converti une chaîne de caractères sur une structure de données Python (comme une liste, un dictionnaire, etc.) en objet réel.# Vérifie si x et une chaîne de caractère
    except:
        return [item.strip() for item in str(x).split(',') if item.strip()] # Supprime les espaces, retours à la ligne, tabulations au début et à la fin d’une chaîne.

df['genres'] = df['genres'].apply(parse_genres)

In [13]:
# Limiter les acteurs aux 5 premiers (pour économiser la RAM)
df['top_actors'] = df['actor_name'].apply(lambda x: x.split(',')[:5] if x else [])
df['top_actors'] = df['top_actors'].apply(lambda x: [item.strip() for item in x if item.strip()])

# Limiter les réalisateurs aux 5 premiers
df['top_directors'] = df['productor_name'].apply(lambda x: x.split(',')[:5] if x else [])
df['top_directors'] = df['top_directors'].apply(lambda x: [item.strip() for item in x if item.strip()])

In [14]:
features_list = []

In [15]:
# 1. Features textuelles (TF-IDF)
if weights['text'] > 0:
    text_data = df['overview'] + ' ' + df['keywords']
    tfidf = TfidfVectorizer(stop_words='english', max_features=3000, max_df=0.8, min_df=2)
    tfidf_matrix = tfidf.fit_transform(text_data)
    tfidf_matrix = tfidf_matrix * weights['text']  # Applique le poids
    features_list.append(tfidf_matrix)
    del text_data    # libère de la mémoire après exécution (ramasse-miettes)
    gc.collect()     # force le ramasse-miettes à faire le ménage immédiatement : il va chercher tous les objets inutilisés en mémoire et les supprimer.

In [16]:
# 2. Features des genres
if weights['genres'] > 0:
    mlb_genres = MultiLabelBinarizer()                                 # MultiLabelBinarizer sert à convertir des listes de labels multiples en format binaire exploitable par les algos de ML, puis à revenir à l'état original si besoin.
    genres_encoded = mlb_genres.fit_transform(df['genres'])            # Transforme le multilabel en matrice binaire
    genres_encoded = csr_matrix(genres_encoded * weights['genres'])    # Stock uniquement les valeurs "1" pour économie de la RAM
    features_list.append(genres_encoded)

In [17]:
# 3. Features des acteurs avec embeddings
if weights['actors'] > 0:                                                           # Oblige à donné une importance
    from sklearn.feature_extraction.text import CountVectorizer
    
    # Convertir les listes d'acteurs en texte
    actors_text = df['top_actors'].apply(lambda x: ' '.join(x) if x else '')
    
    # Vectorisation avec limitation                                                 # # on ne garde que les 200 acteurs les plus fréquents.
    actor_vectorizer = CountVectorizer(max_features=200, binary=True)               # transforme du texte en une matrice binaire ou de fréquences d'apparition des mots
    actors_encoded = actor_vectorizer.fit_transform(actors_text)                    
    actors_encoded = csr_matrix(actors_encoded * weights['actors'])                 # csr_matrix garantit que le résultat reste une matrice creuse (sparse), utile pour l'efficacité mémoire/perf.
    features_list.append(actors_encoded)    # Cette liste sera ensuite fusionnée ou concaténée pour créer une matrice finale pour l'algo

In [18]:
# 4. Features des réalisateurs avec embeddings
if weights['directors'] > 0:
    # Convertir les listes de réalisateurs en texte
    directors_text = df['top_directors'].apply(lambda x: ' '.join(x) if x else '')
    
    # Vectorisation avec limitation
    director_vectorizer = CountVectorizer(max_features=100, binary=True)
    directors_encoded = director_vectorizer.fit_transform(directors_text)
    directors_encoded = csr_matrix(directors_encoded * weights['directors'])
    features_list.append(directors_encoded)

In [19]:
# 5. Features numériques
if weights['numeric'] > 0:
    num_features = df[['budget', 'revenue']].fillna(0)          # Même s'il n'y a plus de NaN
    scaler = MinMaxScaler()                                     # transforme chaque valeur pour qu’elle soit comprise entre 0 et 1 :
    num_scaled = scaler.fit_transform(num_features)             # Le plus petit budget devient 0, le plus grand devient 1, et les autres sont proportionnels. Cela évite que les colonnes à revenu extrême soit trop dominante
    num_scaled = csr_matrix(num_scaled * weights['numeric'])    # num_scaled est maintenant une matrice NumPy de floats normalisés.
    features_list.append(num_scaled)

In [20]:
"""if weights['date'] > 0:
    scaler = MinMaxScaler()
    release_date_scaled = scaler.fit_transform(df[['release_date']])
    release_date_scaled = csr_matrix(release_date_scaled * weights['date'])  # poids faible
    features_list.append(release_date_scaled)"""

"if weights['date'] > 0:\n    scaler = MinMaxScaler()\n    release_date_scaled = scaler.fit_transform(df[['release_date']])\n    release_date_scaled = csr_matrix(release_date_scaled * weights['date'])  # poids faible\n    features_list.append(release_date_scaled)"

In [21]:
# Combiner toutes les features
print("Combinaison des features...")                # features_list contient une liste de matrices sparse
combined_features = hstack(features_list)           # Concatène plusieurs matrices sparse en colonnes, c’est-à-dire côte à côte.
print(f"Shape finale: {combined_features.shape}")   # Le résultat (combined_features) est donc une matrice globale (sparse) avec une ligne par film et toutes les colonnes de toutes les features concaténées
# Cela permet ensuite de :
# - calculer des distances ou similarités entre films,
# - entraîner un modèle de machine learning,
# - ou faire des recherches par voisinage (ex: avec KNN, cosine similarity, etc.).

Combinaison des features...
Shape finale: (29104, 3321)


In [22]:
# Entraîner le modèle
print("Entraînement du modèle...")
nn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_jobs=-1) # trouver les films les plus proches les uns des autres dans l’espace des features
nn_model.fit(combined_features) # mémorise tous les vecteurs de films (1 ligne = 1 film), et être prêt à calculer la distance cosinus

Entraînement du modèle...


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [23]:
# Score de qualité pour boost
quality_features = df[['vote_average', 'popularity', 'vote_count']].fillna(0)
quality_scaler = RobustScaler()                    # met les données à la même échelle, mais de manière robuste
quality_scaled = quality_scaler.fit_transform(quality_features)
quality_score = quality_scaled.mean(axis=1)
quality_score = (quality_score - quality_score.min()) / (quality_score.max() - quality_score.min())
# Contrairement à StandardScaler (qui centre sur la moyenne et réduit selon l'écart-type), ici on centre sur la médiane 
# et on échelle selon l’écart interquartile, ce qui évite d’être influencé aux valeurs extrêmes
print("Prêt pour les recommandations!")

Prêt pour les recommandations!


In [24]:
def get_recommendations(title, top_n=10):
    # Trouver l'index du film
    idx = df[df['title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        return f"Le film '{title}' est introuvable."
    idx = idx[0]
    
    # Rechercher les films similaires
    distances, indices = nn_model.kneighbors(combined_features[idx], n_neighbors=top_n*2)           # attention répétition
    
    # Exclure le film lui-même
    similar_indices = indices.flatten()[1:]
    similarities = 1 - distances.flatten()[1:]
    
    # Créer les résultats
    results = pd.DataFrame({
        'title': df.iloc[similar_indices]['title'].values,
        'similarity': similarities,
        'vote_average': df.iloc[similar_indices]['vote_average'].values,
        'popularity' : df.iloc[similar_indices]['popularity'],
        'quality_score': quality_score[similar_indices]
    })
    
    # Score final : similarité (90%) + qualité (10%)
    results['final_score'] = results['similarity'] * 0.9 + results['quality_score'] * 0.1
    results = results.sort_values('final_score', ascending=False)
    
    return results.head(top_n)[['title', 'similarity', 'vote_average', 'popularity', 'final_score']]

In [31]:
# Test
film_input = input("Entrez le nom d'un film : ")
print(f"\n=== Test avec {film_input} ===")
recommendations = get_recommendations(film_input)
print(recommendations)


=== Test avec The dinner game ===
                         title  similarity  vote_average  popularity  \
5776                 The Valet    0.563416         5.967       9.657   
27774  We Will Go to Deauville    0.517944         5.700       3.081   
6035                The Closet    0.503817         6.401       9.514   
19740            Out on a Limb    0.503843         5.523       5.089   
18762        A Pain in the Ass    0.503855         5.112       3.101   
6850               The ComDads    0.496081         6.581       6.101   
22152          The Lady Banker    0.467647         5.515       3.716   
8604    Three Men and a Cradle    0.451598         6.282       8.749   
8360           Three Fugitives    0.449677         6.658      17.630   
4341            Ruby & Quentin    0.443508         6.717      12.786   

       final_score  
5776      0.508797  
27774     0.467146  
6035      0.455218  
19740     0.454507  
18762     0.454364  
6850      0.448102  
22152     0.421873  
8604